# VacationPy

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Store Part I results into DataFrame

In [ ]:
# Load the CSV exported in Part I to a DataFrame
weather_csv = "../output_data/Weather.csv"
city_weather_df = pd.DataFrame(pd.read_csv(weather_csv))
city_weather_df.head()

# Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# Access maps with unique API key
gmaps.configure(api_key = g_key)

In [ ]:
# Store latitude and longitude in locations
locations = city_weather_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = city_weather_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure(zoom_level = 1.95, center = (20, 5))

# Set max intensity to highest humidity found in the dataset
max_intensity = city_weather_df["Humidity"].max()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating = False, max_intensity = max_intensity,
                                 point_radius = 3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

# Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
# Narrow down the DataFrame to find your ideal weather condition
ideal_conditions_df = city_weather_df.loc[(city_weather_df["Max Temp"] < 80) &
                                          (city_weather_df["Max Temp"] > 70) &
                                          (city_weather_df["Wind Speed"] < 10) &
                                          (city_weather_df["Cloudiness"] == 0), :]

# Drop any rows with null values
ideal_conditions_df.dropna()

# Reset DataFrame index in place
ideal_conditions_df.reset_index(drop = True, inplace = True)

ideal_conditions_df

# Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# Store existing DataFrame into variable named hotel_df using .copy()
hotel_df = ideal_conditions_df[["City","Lat","Lng","Max Temp","Humidity","Cloudiness",
                                "Wind Speed","Country","Date"]].copy()
hotel_df.head()

In [ ]:
# Set up additional column to hold information
hotel_df["Hotel Name"] = ""

hotel_df.head()

In [ ]:
# Set parameters to search for hotels within 5,000 meters
search_keyword = "hotel"
search_radius = 5000

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"keyword": search_keyword,
          "radius": search_radius,
          "key": g_key}

In [ ]:
# Make a request for each of the cities
for index, row in hotel_df[0:10].iterrows():

    # Get coordinates from df
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add coordinates to params dict
    params["location"] = f"{latitude},{longitude}"

    # Assemble url and make API request
    response = requests.get(base_url, params = params).json()
    results = response["results"]
    
    try:
        # Store the first hotel result into the DataFrame
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]

        # Print to display
        print(f"{row['City']}:  closest hotel is {results[0]['name']}.")
    
    except:
        print(f"{row['City']}:  missing field result, skipping...")
    
print("-------------------------------------------------------")

# Create new DataFrame with only 10 results
ten_hotels_df = hotel_df[0:10].copy()
ten_hotels_df

In [ ]:
# Drop any rows with null values
ten_hotels_df.dropna(axis = 0, how = "any", inplace = True)

# Display updated DataFrame
ten_hotels_df.head()

In [ ]:
ten_hotels_df.isnull().sum()

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ten_hotels_df.iterrows()]
plot_locations = ten_hotels_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer on top of heat map
plot_markers = gmaps.marker_layer(plot_locations, info_box_content = hotel_info)

# Plot Heatmap
fig2 = gmaps.figure(zoom_level = 1.95, center = (20, 5))

# Set max intensity to highest humidity found in the dataset
max_intensity = city_weather_df["Humidity"].max()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating = False, max_intensity = max_intensity,
                                 point_radius = 3)

# Add layers
fig2.add_layer(heat_layer)
fig2.add_layer(plot_markers)

# Display figure
fig2